In [4]:
from chat_tools import get_completion_from_messages, get_completion_and_token_count

In [31]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import utils_zh

_ = load_dotenv(find_dotenv())

def process_user_message_ch(user_input, all_messages, debug=True):
    """
    对用户信息进行预处理
    
    参数:
    user_input : 用户输入
    all_messages : 历史信息
    debug : 是否开启 DEBUG 模式,默认开启
    """
    delimiter = "```"

    client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
    )

    response = client.moderations.create(input=user_input)
    moderation_output = response.results[0]
    # print(moderation_output)

    if moderation_output.flagged:
        print("第一步：输入被 Moderation 拒绝")
        return "抱歉，您的请求不合规"
    
    # 如果开启了 DEBUG 模式，打印实时进度
    if debug: print("第一步：输入通过 Moderation 检查")

    # print(utils_zh.get_products_and_category())
    category_and_product_response = utils_zh.find_category_and_product_only(user_input, utils_zh.get_products_and_category())
    # print(category_and_product_response)

    # 将抽取出来的字符串转化为列表
    category_and_product_list = utils_zh.read_string_to_list(category_and_product_response)
    # print(category_and_product_list)
    if debug: print("第二步：抽取出商品列表")

    # 第三步：查找商品对应信息
    product_information = utils_zh.generate_output_string(category_and_product_list)
    # print(product_information)
    if debug: print("第三步：查找抽取出的商品信息")

    # 第四步：根据信息生成回答
    system_message = f"""
        您是一家大型电子商店的客户服务助理。\
        借助商品信息回答客户提问。\
        请以友好和乐于助人的语气回答问题，并提供简洁明了的答案。\
        请确保向用户提出相关的后续问题。
    """
    # 插入 message
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"相关商品信息:\n{product_information}"}
    ]
    # 获取 GPT3.5 的回答
    # 通过附加 all_messages 实现多轮对话
    final_response = get_completion_from_messages(all_messages + messages)
    # print(final_response)
    if debug:print("第四步：生成用户回答")
    # 将该轮信息加入到历史信息中
    # print(messages[1:])
    all_messages = all_messages + messages[1:]

    # 第五步：基于 Moderation API 检查输出是否合规
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]
    # 输出不合规
    if moderation_output.flagged:
        if debug: print("第五步：输出被 Moderation 拒绝")
        return "抱歉，我们不能提供该信息"
    
    if debug: print("第五步：输出经过 Moderation 检查")

    # 第六步：模型检查是否很好地回答了用户问题
    user_message = f"""`
    顾客的信息: {delimiter}{user_input}{delimiter}
    代理的回复: {delimiter}{final_response}{delimiter}
    回复是否正确回答了客户的提问？
    若赞同输出 Y，不赞同输出N
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    # 要求模型评估回答
    evaluation_response = get_completion_and_token_count(messages, max_tokens=1)
    if debug: print("第六步：模型评估该回答")
    # print(evaluation_response)

    # 第七步：如果评估为 Y，输出回答；如果评估为 N，反馈将由人工修正答案
    if "Y" in evaluation_response:  # 使用 in 来避免模型可能生成 Yes
        if debug: print("第七步：模型赞同了该回答.")
        return final_response, all_messages
    else:
        if debug: print("第七步：模型不赞成该回答.")
        neg_str = "很抱歉，我无法提供您所需的信息。我将为您转接到一位人工客服代表以获取进一步帮助。"
        return neg_str, all_messages

    # print(evaluation_response)

In [32]:
user_input = "请告诉我关于 smartx pro phone 和 the fotosnap camera 的信息。另外，请告诉我关于你们的tvs的情况。"
response,_ = process_user_message_ch(user_input,[])
print(response)

第一步：输入通过 Moderation 检查
第二步：抽取出商品列表
第三步：查找抽取出的商品信息
第四步：生成用户回答
第五步：输出经过 Moderation 检查
第六步：模型评估该回答
第七步：模型赞同了该回答.
关于您提到的商品信息：

1. **SmartX ProPhone**:
   - 价格: $899.99
   - 特色: 6.1英寸显示屏、128GB存储、12MP双摄像头、5G网络
   - 描述: 具有先进摄像功能的强大智能手机

2. **FotoSnap DSLR Camera**:
   - 价格: $599.99
   - 特色: 24.2MP传感器、1080p视频、3英寸LCD、可更换镜头
   - 描述: 使用这款多功能的单反相机，捕捉惊艳的照片和视频

3. **电视情况**:
   - 我们有多款电视可供选择，包括CineView 4K TV、CineView 8K TV和CineView OLED TV，价格从$599.99到$2999.99不等。
   - 每款电视都具有不同的特色，如4K或8K分辨率、HDR和智能电视功能。
   
您对这些商品有进一步的问题或需要了解其他商品吗？
